Importing Standard Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
import re
from PIL import Image
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

EDA & Feature Engineering

In [ ]:
df_250 = pd.read_csv('../input/restaurant-business-rankings-2020/Top250.csv')
df_future = pd.read_csv('../input/restaurant-business-rankings-2020/Future50.csv')
df_ind = pd.read_csv('../input/restaurant-business-rankings-2020/Independence100.csv')

In [ ]:
df_250.info()

In [ ]:
df_250.drop(['Content','Headquarters'],axis=1,inplace=True)

In [ ]:
print ("Rows     : " ,df_250.shape[0])
print ("Columns  : " ,df_250.shape[1])
print ("\nFeatures : \n" ,df_250.columns.tolist())
print ("\nMissing values :  ", df_250.isnull().sum().values.sum())
print ("\nUnique values :  \n",df_250.nunique())

In [ ]:
df_250.rename(columns={'Units':'Branches','Segment_Category':'sub_category'},inplace=True)

In [ ]:
rest_ind = list(df_ind['Restaurant'].values)
df_250['ind_100']=df_250['Restaurant'].apply(lambda x:1 if x in rest_ind else 0 )
df_250['ind_100'].value_counts()

In [ ]:
df_250['YOY_Sales']=df_250['YOY_Sales'].apply(lambda x:x.replace('%',''))
df_250['YOY_Units']=df_250['YOY_Units'].apply(lambda x:x.replace('%',''))
#====
df_250['YOY_Sales']= pd.to_numeric(df_250['YOY_Sales'])
df_250['YOY_Units']=pd.to_numeric(df_250['YOY_Units'])
#====

In [ ]:
df_250.describe()

In [ ]:
df_250['Sales_year']=df_250['YOY_Sales'].apply(lambda x:'positive' if x>0 else 'negative')
df_250['unit_stat']=df_250['YOY_Units'].apply(lambda x:'positive' if x>0 else 'negative')

Making Category Feature

In [ ]:
df_250['category']=df_250['sub_category'].apply(lambda x:x.replace('Quick Service & Burger','Burger').replace(
                                                      'Italian/Pizza','Pizza').replace('Quick Service & Pizza','Pizza').replace(
'Bakery Cafe ','Cafe').replace('Coffee Cafe','Cafe')).replace('Quick Service & Chicken','Chicken').replace('Casual Dining & Pizza','Pizza').replace(
'Quick Service & Cafe','Cafe').replace('Fast Casual & Pizza','Pizza').replace('Fast Casual & Bakery Cafe','Cafe').replace('Quick Service & Frozen Desserts','Dessert').replace(
'Quick Service & Family Casual','Family').replace('Casual Dining & Asian','Asian').replace('Asian/Noodle','Asian').replace('Casual Dining & Seafood','Seafood').replace(
'Bakery Cafe','Cafe').replace('Frozen Desserts','Dessert').replace('Family Dining & Family Style','Family').replace('Fast Casual & Sandwich','Sandwich').replace('Family Casual','Family').replace('Fast Casual & Chicken','Chicken').replace('Fast Casual & Burger','Burger').replace('Casual Dining & Steak','Steak').replace(
'Casual Dining & Sports Bar','Sports Bar').replace('Quick Service & Mexican','Mexican').replace('Quick Service & Mexican','Mexican').replace(
'Quick Service & Seafood','Seafood').replace('Quick Service & Sandwich','Sandwich').replace('Seafodd','Seafood').replace('Fine Dining & Steak','Steak').replace(
'Quick Service & Bakery Cafe','Cafe').replace('Fast Casual & Asian/Noodle','Asia').replace('Quick Service & Snack','Snack').replace(
'Fast Casual & Mexican','Mexican').replace('Quick Service & Beverage','Drinks').replace('Asian','Asia').replace('Sports Bar','Sports Bar').replace(
'Casual Dining & Varied Menu','Varied Menu').replace('Steak','Meat').replace('BBQ','Meat').replace('Family Style','Family')

In [ ]:
df_250.head()

Data Visualization

YOY Distribution

In [ ]:
plt.figure(figsize=(14,7))
sns.kdeplot(df_250['YOY_Sales'].values, shade = True)
sns.kdeplot(df_250['YOY_Units'].values, shade = True)
plt.legend(['Sales','Units'])
plt.xlabel('Percentage');

These Distributions shows us that year on year sales percentage no more than 40%

**Correlation with sales**

In [ ]:
corr = df_250.corr().loc[:,['Sales']]

fig, ax = plt.subplots(figsize=(5,10))

ax = sns.heatmap(corr.sort_values(by=['Sales'],ascending=False), ax=ax,annot=True,cbar=True,cmap="Greens")
ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=10)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=10)

plt.show()

Sales is highly correlated with branches

YOY Sales Indicator

In [ ]:
plt.figure(figsize=(14,7));
Platform = df_250['Sales_year'].value_counts(sort = True).index
Count = df_250['Sales_year'].value_counts(sort = True)
#====
fig = px.pie(names = Platform,
             values = Count,
             title='Positive - Negative')
fig.update_traces(textposition='inside', textinfo='percent+label')

Although the restaurants were onthe list for the best restaurants of the year, about 35% of restaurants had negative indicators

Restaurant Category and Sub Category

In [ ]:
plt.figure(figsize=(40,40));

df = px.data.tips()
fig = px.sunburst(df_250, path=['category', 'sub_category'], values='Sales')
fig.show()

sunburst shows that quick-service subcategory always get high sales

Category Sales Indicator

In [ ]:
plt.figure(figsize=(20,7))
#=== 
pal = ["#58D3F7","#FA5858"]
#=== 
sns.barplot(data=df_250,x=df_250['category'],y=df_250['Sales'],hue=df_250['Sales_year'],palette=pal);

varied menu, sandwiches and sports bar had more negative indicators than positive

Top Resturants by food category

Pizza

In [ ]:
pizza = df_250.query("category == 'Pizza'")[:10]

fig = px.bar(pizza,
             x='Restaurant',
             y='Sales',
             color='Sales',
             title='Pizza Sales Top Restaurants')
fig.show()

Burger

In [ ]:
Burger = df_250.query("category == 'Burger'")[:10]

fig = px.bar(Burger,
             x='Restaurant',
             y='Sales',
             color='Sales',
             title='Burger Top Restaurants')
fig.show()

Varied Menu

In [ ]:
VariedMenu = df_250.query("category == 'Varied Menu'")[:10]

fig = px.bar(VariedMenu,
             x='Restaurant',
             y='Sales',
             color='Sales',
             title='VariedMenu Top Restaurants')
fig.show()